In [16]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, autojit, prange
# Disable inline plots
%matplotlib qt

In [2]:
# Pad dataset before filter based on window provided
# Give data set (d_set), and window width (x)
# Needs numpy as np
@jit
def winpad(d_set, x):
    if (x % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    rad = np.int(np.floor(x/2)) # Calculate radius of window given
    pad = np.pad(d_set, rad, 'edge')
    return pad
# Optimized modified 3D hampel filter
# Runs in parallel on CPU
# Removes nans from center region of volume that corresponds to ~10% the volume width and replaces with zero
# Only applies filter to voxels with finite values 
# Uses a cube moving window, center point in window is checked
# Returns mean/median value that has the least difference between the original value for outliers
# Input dataset (dset) and window width (x)
# Input an odd window or the window will be asymmetric and stuff breaks

from numba import autojit, prange
@jit(parallel=True)
def mod_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mean = np.mean(dat) # Get mean of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        if np.abs(mean-val) > np.abs(med-val):
            out[x, y, z] = med # Return median if abs(med-val) is smaller than abs(mean-val)
            continue
        else:
            out[x, y, z] = mean # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out
# This filter makse weird negative correlations after the FT for some reason where they shouldn't be
# Don't use
# Optimized 3D hampel filter
# Runs in parallel on CPU
# Removes nans from center region of volume that corresponds to ~10% the volume width and replaces with zero
# Only applies filter to voxels with finite values 
# Uses a cube moving window, center point in window is checked
# Returns median value for outliers
# Input dataset (dset) and window width (x)
# Input an odd window or the window will be asymmetric and stuff breaks

from numba import autojit, prange
@jit(parallel=True)
def norm_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mean = np.mean(dat) # Get mean of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = med # Return median if value is outlier
            continue
    
    return out
# Does a 1D ft, doesn't need to be rolled over
@jit
def ft(signal):
    return np.fft.fftshift(np.fft.fftn(np.fft.ifftshift(signal)))
# Fourier transform 2D slices of an image
@jit
def ft_slices(data):
    shape = np.shape(data) # Get shape of input data
    fslices = np.zeros(shape, dtype = np.complex_) # Array of complex zeros with same shape as input data
    l1 = shape[0] # Number of 2D slices through 3D volume
    for i in range (0, l1):
        fslices[i] = ft(data[i]) # Assign FT of individual slices to corresponding slice in zero array
        #print(i, end = ' ') # Testing counter printout
    return fslices
# Take FT of columns through 2D slices
@jit
def ft_columns(data):
    shape = np.shape(data) # Get shape of input data
    out = np.zeros(shape, dtype = np.complex_) # Array of complex zeros with same shape as input data
    l1 = shape[0] # Total number of loops for y dim
    l2 = shape[1] # Total number of loops for z dim
    for i in range (0, l1): # Loop through y
        for j in range (0, l2): # Loop through z
            out[:, i, j] = ft(data[:, i, j]) # Assign FT through x axis to corresponding region of zero array
        #print (i, end = ' ') # Testing counter printout
    return out
# Take 3D FT by taking FT of 2D slices through an image and then 1D columns perpendicular to slices
@jit
def ft_3d(data):
    slices = ft_slices(data)
    cols = ft_columns(slices)
    return cols

In [3]:
# No significant difference seen with original modified hampel other than slightly lower SNR
# Returns mean of window excluding checked value for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod2_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mean = np.mean(np.concatenate((dat[0:winrad], dat[winrad+1:]))) # Get mean of input data set without center value
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        if np.abs(mean-val) > np.abs(med-val):
            out[x, y, z] = med # Return median if abs(med-val) is smaller than abs(mean-val)
            continue
        else:
            out[x, y, z] = mean # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out

# Lower SNR for positive correlations, don't use
# Returns 2x median of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod3_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = 3*med # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out

# Creates weird negative correlations, do not use
# Returns 2x value for non outliers
# Returns mean for outliers
from numba import autojit, prange
@jit(parallel=True)
def mod4_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mean = np.mean(dat) # Get mean of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = 2*val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = mean # Return median if value is outlier
            continue
    
    return out
# Improved SNR over mod1
# Returns 3x median of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod5_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = 3*med # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out
# Improved SNR over mod1, lower SNR than mod5
# Returns median+(mad*3*1.4826) of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod6_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = med+asigma # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out
# Slightly worse than mod5
# Returns median+2*(mad*3*1.4826) of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod7_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = med+2*asigma # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out

# Very slightly worse than mod5
# Returns median+4*(mad*1.4826) of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod8_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = med+4*mad*1.4826 # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out

# Marginally better than mod5
# Returns median+3.5*(mad*1.4826) of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod9_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = med+3.5*mad*1.4826 # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out

# 
# Returns median+3.5*(mad*1.4826) of window for values detected to be outliers
from numba import autojit, prange
@jit(parallel=True)
def mod10_3d_hampel(d_set, width):
    if (width % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    shape = np.shape(d_set) # Get shape of input data set
    winrad = np.floor(np.divide(width, 2)) # Window radius
    out = np.zeros(shape) # Generate set of zeros same shape as input data set
    # Remove nans from center of image
    # This might not actually be necessary
    rad = (shape[0]/2)*0.1 # Radius for center region to remove nans from, ~10% of total volume width
    a, b = shape[0]/2-rad, shape[0]/2+rad # Bounds for center region to remove nans
    a=int(a) # Change to int values
    b=int(b) 
    d_set[a:b, a:b, a:b] = np.nan_to_num(d_set[a:b, a:b, a:b]) # Remove nans from center of volume
    x_pos, y_pos, z_pos = np.where(np.isfinite(d_set)) # Get location of finite values (pixels to be filtered)
    # Remove nans and very large positive/negative values before filtering
    d_set = np.nan_to_num(d_set)
    d_set[d_set>1e+100]=0
    d_set[d_set<1e-100]=0
    # Create padded data set to avoid windowing issues with values at edge
    temp = winpad(d_set, width)
    # Get loop length by taking length of list of pixels to be filtered
    length = len(x_pos)
    # Apply filter over pixels
    for i in prange (0, length):
        x, y, z = x_pos[i], y_pos[i], z_pos[i]
        # Outlier detection, basically the same as is_outlier_mm function
        # Only here to reduce overhead from function calls
        dat = temp[x:x+width, y:y+width, z:z+width]
        val = d_set[x, y, z]
        med = np.median(dat) # Get median of input data set
        mad = np.median(np.abs(dat- med)) # Get median absolute deviation
        asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
        if (med-asigma) < val < (med+asigma): # Check if value is outlier based on MAD
            out[x, y, z] = val # Return value if it is not an outlier
            continue
        else:
            out[x, y, z] = np.divide((3*med)+(med+3.5*mad*1.4826), 2) # Return mean if abs(med-val) is l arger than abs(mean-val)
            continue
    
    return out

In [47]:
g = np.load('gaussian window function.npy')
pmn = np.load('benzil_300K_bkg_subtract_sym_All_noCC_flat2.npy')
pmn = np.multiply(g, pmn)

In [48]:
#fpmn = mod_3d_hampel(pmn, 3)
#fpmn2 = mod3_3d_hampel(pmn, 11)
#htest = norm_3d_hampel(pmn, 5)
#ftest = mod9_3d_hampel(pmn, 5)
#mod5 = mod5_3d_hampel(pmn, 5)
mod9 = mod9_3d_hampel(pmn, 5)
#mod95 = mod10_3d_hampel(pmn, 5)
#mod97 = mod10_3d_hampel(pmn, 7)
#mod99 = mod10_3d_hampel(pmn, 9)
#mod911 = mod10_3d_hampel(pmn, 11)
#mod10 = mod10_3d_hampel(pmn, 5)
#nhamp = norm_3d_hampel(pmn, 5)
#test = mod9_3d_hampel(pmn, 11)
#test2 = mod10_3d_hampel(pmn, 5)

In [7]:
fpmn = np.load('filtered_pmn_no_interpolation_recalculated.npy')

In [157]:
plt.imshow(ftest2[250])

In [92]:
from skimage.transform import rotate

In [172]:
# Rotate 45 degrees to align row with diffuse
# View row 302
# Unfiltered data
a = rotate(pmn[250], 45)[302]
# Filtered data
b = rotate(ftest[250], 45)[302]
c = rotate(test2[250], 45)[302]

plt.plot(a, 'r', b, 'g', c, 'b')

C:\Users\6jw\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:29: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims)
C:\Users\6jw\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)


In [163]:
plt.imshow(rotate(fpmn[250], 45))

In [173]:
plt.subplot(1, 2, 1)
#plt.subplot(1, 3, 1)
plt.imshow(ftest[250], cmap = 'viridis')
plt.subplot(1, 2, 2)
#plt.subplot(1, 3, 2)
plt.imshow(test2[250], cmap = 'viridis')
#plt.subplot(1, 3, 3)
#plt.imshow(fpmn3[250], cmap = 'viridis')

In [ ]:
np.max(fnpmn)

In [126]:
plt.plot(pmn[250][250], 'r', ftest[250][250], 'g', mod5[250][250], 'b')

In [124]:
plt.plot(mf[250][250], 'r', htest[250][250], 'g', ftest[250][250], 'b')

In [49]:
#ref = np.load('pmn_3d_patterson_recalculated.npy')
#ft2 = ft_3d(ftest)
#ft1 = ft_3d(fpmn)
#m5ft = ft_3d(mod5)
m9ft = ft_3d(mod9)
#m95ft = ft_3d(mod95)
#m97ft = ft_3d(mod97)
#m99ft = ft_3d(mod99)
#m911ft = ft_3d(mod911)
#m10ft = ft_3d(mod10)
#mft1 = ft_3d(test)
#mft2 = ft_3d(ftest)


In [50]:
mask = np.ones((501,501, 501))
mask[248:253, 248:253, 248:253] = 0

In [51]:
#t1 = np.multiply(ft1, mask)
#ft2 = np.multiply(ft2, mask)
#ft3 = np.multiply(ft3, mask)
#mft = np.multiply(mft, mask)
#mft1 = np.multiply(mft1, mask)
#mft2 = np.multiply(mft2, mask)
#ef = np.multiply(ref, mask)
#m5ft = np.multiply(m5ft, mask)
m9ft = np.multiply(m9ft, mask)
#m95ft = np.multiply(m95ft, mask)
#m97ft = np.multiply(m97ft, mask)
#m99ft = np.multiply(m99ft, mask)
#m911ft = np.multiply(m911ft, mask)
#m10ft = np.multiply(m10ft, mask)

In [52]:
np.save('benzil 300K, mod_9_hampel.npy', np.real(m9ft))
np.save('benzil 300K, mod_9_hampel, cropped.npy', np.real(m9ft)[200:300, 200:300, 200:300])

In [14]:
from skimage import exposure

In [18]:
a = np.real(ref)[250]
b = np.real(m9ft)[250]
#c = np.real(ft3)[250]
plt.subplot(1, 2, 1)
plt.imshow(a, cmap = 'magma')
plt.subplot(1, 2, 2)
plt.imshow(b, cmap = 'magma')
#plt.subplot(1, 3, 3)
#plt.imshow(exposure.equalize_hist(c), cmap = 'magma')

In [66]:
plt.imshow(np.real(ft2)[250])

In [26]:
a = np.real(m911ft)[250]
#c = np.real(ft3)[250]
plt.imshow(a, cmap = 'coolwarm')
plt.title('modified hampel filter 9, moving window 11x11x11')
plt.colorbar()

In [47]:
a = np.real(m9ft)[250][270]
b = np.real(m95ft)[250][270]
c = np.real(m97ft)[250][270]
d = np.real(m99ft)[250][270]
e = np.real(m911ft)[250][270]
#c = np.real(ft3)[250][270]
red_patch = mpatches.Patch(color = 'red', label = '3x3x3')
green_patch = mpatches.Patch(color = 'green', label = '5x5x5')
blue_patch = mpatches.Patch(color = 'blue', label = '7x7x7')
plt.legend(handles = [red_patch, green_patch, blue_patch])
plt.title('Slice through 3d patterson, [250][270]')
plt.plot(a, 'b', b, 'r', c, 'g')

In [41]:
a = np.real(m9ft)[250][270]
b = np.real(m95ft)[250][270]
a = np.divide(a, np.max(a))
b = np.divide(b, np.max(b))
#c = np.real(ft3)[250][270]
red_patch = mpatches.Patch(color = 'red', label = '3x3x3')
green_patch = mpatches.Patch(color = 'green', label = '5x5x5')
plt.legend(handles = [red_patch, green_patch])
plt.title('Slice through 3d patterson, [250][270]')
plt.plot(a, 'r', b, 'g')

In [19]:
a = np.real(ref)[250]
b = np.real(m9ft)[250]
#c = np.real(ft3)[250]
plt.subplot(1, 2, 1)
plt.imshow(a, cmap = 'coolwarm')
#plt.title('modified hampel filter 1')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(b, cmap = 'coolwarm')
#plt.title('modified hampel filter 9')
plt.colorbar()
#plt.subplot(1, 3, 3)
#plt.imshow(c, cmap = 'viridis')

In [28]:
import matplotlib.patches as mpatches

In [17]:
a = np.real(ref)[250][270]
b = np.real(m9ft)[250][270]
#c = np.real(ft3)[250][270]
red_patch = mpatches.Patch(color = 'red', label = 'modified hampel 1')
green_patch = mpatches.Patch(color = 'green', label = 'modified hampel 9')
plt.legend(handles = [red_patch, green_patch])
plt.title('Slice through 3d patterson, [250][270]')
plt.plot(a, 'r', b, 'g')#, c, 'b')

In [21]:
np.save('pmn, mod_9_hampel.npy', np.real(m9ft))
np.save('pmn, mod_9_hampel, cropped.npy', np.real(m9ft)[200:300, 200:300, 200:300])

In [148]:
plt.plot(fpmn[250][250], 'r', ftest[250][250], 'g')
plt.plot(fpmn[250][250], 'r', ftest[250][250], 'g', marker = '.')

In [87]:
fpmn = np.load('filtered_pmn_no_interpolation_recalculated.npy')

In [82]:
import time
start = time.time()
rpmn = mod_3d_hampel(pmn, 9)
stop = time.time()
print (stop-start)

255.91649913787842


In [97]:
timeit(mod_3d_hampel(pmn, 11))

7min 15s ± 2.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
test1 = pmn[225:275, 225:275, 225:275]
test2 = fpmn[225:275, 225:275, 225:275]

In [69]:
test3 = mod_3d_hampel(test1,21)

In [46]:
np.sum(rpmn[249])

0.0

In [84]:
plt.plot(rpmn[250][250], 'r', fpmn[250][250], 'g')

In [83]:
plt.imshow(rpmn[250])

In [85]:
ft2 = ft_3d(rpmn)

In [95]:
plt.plot(ft1[250][240], 'r', ft2[250][240], 'g')

C:\Users\6jw\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [90]:
plt.imshow(exposure.equalize_hist(np.real(ft2[250])))

In [89]:
plt.imshow(np.real(ft2[250]), cmap = 'coolwarm')

In [24]:
plt.plot(test[245][250], 'r', fpmn[245][250], 'g')

In [8]:
dumb = ft_3d(test)

In [88]:
ft1 = ft_3d(fpmn)
#ft2 = ft_3d(f2pmn)
#ft3 = ft_3d(fpmn2)

In [22]:
#plt.plot(ft1[250][250], 'r', ft2[250][250], 'g',  ft3[250][250], 'b')
plt.plot(ft1[250][250], 'r', dumb[250][250], 'g')

C:\Users\6jw\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [21]:
plt.imshow(exposure.equalize_hist(np.real(dumb[250])))

In [90]:
ft1 = np.multiply(ft1, mask)
#ft2 = np.multiply(ft2, mask)
#ft3 = np.multiply(ft3, mask)
dumb = np.multiply(dumb, mask)

In [17]:
plt.subplot(1, 2, 1)
#plt.subplot(1, 3, 1)
plt.imshow(np.real(ft1[250]), cmap = 'magma')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(np.real(dumb[250]), cmap = 'magma')
plt.colorbar()
#plt.subplot(1, 3, 2)
#plt.imshow(np.real(ft2[250]), cmap = 'magma')
#plt.colorbar()
#plt.subplot(1, 3, 3)
#plt.imshow(np.real(ft3[250]), cmap = 'magma')
#plt.colorbar()

In [150]:
dumb = np.multiply(dumb, mask)

In [67]:
plt.imshow(np.real(dumb[250]), cmap = 'magma')

In [91]:
np.save('pmn_3d_patterson_center_mask_real, cropped.npy', np.real(ft1)[200:300, 200:300, 200:300])

In [86]:
plt.imshow(np.real(ft1)[250])

In [79]:
dto = np.load('dto_H1.2T_patterson.npy')

In [94]:
np.save('dto_H1.2T_patterson_real, cropped', np.multiply(np.real(dto), mask)[200:300,200:300,200:300])

In [96]:
benz = np.load('benzil_300k_3d_patterson.npy')

In [97]:
np.save('benzil_300k_patterson_real, cropped', np.multiply(np.real(benz), mask)[200:300,200:300,200:300])